In [1]:
import sys
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

sys.path.append('../')

### Binary

In [2]:
train = pd.read_parquet('/www/dslib/spark_sota_modeling/dataset/home-credit-default-risk/train.parquet')
target_col = 'target'
index_col = 'sk_id_curr'
train, test = train_test_split(train, test_size=0.2, random_state=42, stratify=train[target_col])
X_train = train.drop(columns=[target_col, index_col])
y_train = train[target_col]
X_test = test.drop(columns=[target_col, index_col])
y_test = test[target_col]

In [3]:
from models.nn.tabnet import TabNetBinaryClassifier

model = TabNetBinaryClassifier(
    verbose=True,
    scale_method='standard', # standard, minmax, quantile, binning
    dropout=0.6,
    lambda_sparse=0.001,
    learning_rate=0.0015,
    epochs=200,
    early_stopping_patience=10,
)

model.fit(X_train, y_train, eval_set=(X_test, y_test), eval_metric='roc_auc', mode='max')

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

Начинаем обучение на cuda...


Training:   0%|          | 0/241 [00:00<?, ?it/s]

Validation:   0%|          | 0/61 [00:00<?, ?it/s]

Epoch 1/200
Train loss: 0.3806, Train roc_auc: 0.5894
Val loss: 0.2564, Val roc_auc: 0.7512
Сохраняем лучшую модель с метрикой roc_auc: 0.7512


Training:   0%|          | 0/241 [00:00<?, ?it/s]

...

Training:   0%|          | 0/241 [00:00<?, ?it/s]

Validation:   0%|          | 0/61 [00:00<?, ?it/s]

Epoch 57/200
Train loss: 0.2374, Train roc_auc: 0.7886
Val loss: 0.2432, Val roc_auc: 0.7723
Нет улучшения в течение 10 эпох
Останавливаем обучение из-за отсутствия улучшений
Загружена лучшая модель


Predicting:   0%|          | 0/61 [00:00<?, ?it/s]

Predicting:   0%|          | 0/61 [00:00<?, ?it/s]

In [4]:
roc_auc_score(y_test, y_pred_proba[:,1])

0.7723695622799762

In [ ]:
# standard  0.7737342479104056
# minmax:   0.7629154604000974
# quantile: 0.7729699391727092
# binning:  0.7630777980797842

### Multiclass

In [2]:
train = pd.read_parquet('/www/dslib/spark_sota_modeling/dataset/forest-cover-type/train.parquet')
target_col = 'cover_type'
train, test = train_test_split(train, test_size=0.2, random_state=42, stratify=train[target_col])
X_train = train.drop(columns=[target_col])
y_train = train[target_col]
X_test = test.drop(columns=[target_col])
y_test = test[target_col]

In [3]:
from models.nn.tabnet import TabNetMulticlassClassifier

# cat_emb_dim=6,  # Размерность эмбеддингов для категориальных признаков
# n_steps=4,  # Количество шагов в TabNet
# hidden_dim=16,  # Размерность скрытого слоя
# decision_dim=8,  # Размерность решающего слоя
# n_glu_layers=3,  # Количество GLU слоев
# dropout=0.6,  # Вероятность дропаута
# gamma=1.5,  # Коэффициент затухания для масок внимания
# lambda_sparse=0.0001,  # Коэффициент регуляризации разреженности
# virtual_batch_size=128,  # Размер виртуального батча для Ghost BatchNorm
# momentum=0.9,  # Параметр momentum для BatchNorm
# batch_size=1024,  # Размер батча для обучения
# epochs=50,  # Количество эпох обучения
# learning_rate=0.005,  # Скорость обучения
# early_stopping_patience=5,  # Количество эпох без улучшения до остановки
# weight_decay=1e-5,  # Весовая регуляризация для оптимизатора
# scale_numerical=True,  # Масштабировать ли числовые признаки
# scale_method="standard",  # Метод масштабирования ("standard", "minmax", "quantile", "binning")
# n_bins=10,  # Количество бинов для binning

model = TabNetMulticlassClassifier(
    verbose=True,
    n_classes=train[target_col].nunique(),
    hidden_dim=64,
    decision_dim=32,
    n_steps=5,
    n_glu_layers=3,
    dropout=0.1,
    gamma=1.5,
    lambda_sparse=0.0001,
    batch_size=16384,
    virtual_batch_size=512,
    momentum=0.7,
    learning_rate=0.05,
    epochs=1000,
    cat_emb_dim=6,
    early_stopping_patience=30,
    # scale_method='standard', # standard, minmax, quantile, binning
)

model.fit(X_train, y_train, eval_set=(X_test, y_test))

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

Начинаем обучение на cuda...


Training:   0%|          | 0/29 [00:00<?, ?it/s]

Validation:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1/1000
Train loss: 0.7784, Train accuracy: 0.6773
Val loss: 0.6024, Val accuracy: 0.7429
Сохраняем лучшую модель с метрикой accuracy: 0.7429


Training:   0%|          | 0/29 [00:00<?, ?it/s]

...

Training:   0%|          | 0/29 [00:00<?, ?it/s]

Validation:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 141/1000
Train loss: 0.1977, Train accuracy: 0.9182
Val loss: 0.1570, Val accuracy: 0.9368
Нет улучшения в течение 20 эпох
Останавливаем обучение из-за отсутствия улучшений
Загружена лучшая модель


Predicting:   0%|          | 0/8 [00:00<?, ?it/s]

Predicting:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9375403388897017

In [ ]:
# 0.8479471270104902
# 0.9272996394241112

### Regression

In [2]:
train = pd.read_parquet('/www/dslib/spark_sota_modeling/dataset/allstate-claims-severity/train.parquet')
target_col = 'loss'
index_col = 'id'
train, test = train_test_split(train, test_size=0.2, random_state=42)
X_train = train.drop(columns=[target_col, index_col])
y_train = train[target_col]
X_test = test.drop(columns=[target_col, index_col])
y_test = test[target_col]

In [6]:
from models.nn.tabnet import TabNetRegressor
model = TabNetRegressor(
    verbose=True,
    dropout=0.3,
    lambda_sparse=0.001,
    learning_rate=0.0015,
    epochs=200,
    early_stopping_patience=20,
)

model.fit(X_train, y_train, eval_set=(X_test, y_test), eval_metric='mae', mode='min')

y_pred = model.predict(X_test)

Начинаем обучение на cuda...


Training:   0%|          | 0/148 [00:00<?, ?it/s]

Validation:   0%|          | 0/37 [00:00<?, ?it/s]

Epoch 1/200
Train loss: 17728274.3446, Train mae: 3038.4465
Val loss: 17263758.3514, Val mae: 3024.4260
Сохраняем лучшую модель с метрикой mae: 3024.4260


Training:   0%|          | 0/148 [00:00<?, ?it/s]

...

Epoch 50/200
Train loss: 3765392.2145, Train mae: 1234.2762
Val loss: 3663335.9865, Val mae: 1204.0587
Нет улучшения в течение 19 эпох


Training:   0%|          | 0/148 [00:00<?, ?it/s]

Validation:   0%|          | 0/37 [00:00<?, ?it/s]

Epoch 51/200
Train loss: 3733833.7162, Train mae: 1230.9913
Val loss: 3873510.2162, Val mae: 1180.6818
Нет улучшения в течение 20 эпох
Останавливаем обучение из-за отсутствия улучшений
Загружена лучшая модель


Predicting:   0%|          | 0/37 [00:00<?, ?it/s]

In [7]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

1173.9303179279098

In [ ]:
# 1173.9303179279098